In [1]:
import requests
from requests.structures import CaseInsensitiveDict
import json
import numpy as np
from strava_key_gen import access_token
from typing import Dict, Tuple
import pandas as pd
import polyline
from geopy import distance

id = 5270481


def return_segment(id: int, access_token: str) -> json:
    url = f"https://www.strava.com/api/v3/segments/{id}"
    headers = CaseInsensitiveDict()
    headers["Authorization"] = f"Bearer {access_token}"
    resp = requests.get(url, headers=headers)
    return resp.json()


def get_params_from_segment_from_json(segment):
    return segment['name'], segment['distance'], segment['average_grade'], segment['maximum_grade'], polyline.decode(segment['map']['polyline']) 

def retrieve_segment(id: int, access_token:str):
    seg_json = return_segment(id, access_token)
    name, distance, average_grade, maximum_grade, lat_lon = get_params_from_segment_from_json(seg_json)
    return name, distance, average_grade, maximum_grade, lat_lon 

def build_api_call_string(lat_lon):
    length = len(lat_lon)
    iterations =int(np.ceil(length/100))
    start, end = 0, 99
    string_lists = []
    for i in range(iterations):
        string = ''
        for j in range(start, end):
            lat, lon = lat_lon[j][0], lat_lon[j][1]
            string += str(lat) + ',' + str(lon)+ '|'
        string_lists.append(string[:-1]) # drop the last pipe
        start += 100
        
        if end + 100 > len(lat_lon):
            end = len(lat_lon) 
        else:
            end += 100
    
    return string_lists


def get_elevations_lat_lon(string: str):
    url = f"https://api.opentopodata.org/v1/eudem25m?locations={string}"
    resp = requests.get(url)
    resp = resp.json()['results']
    result_list = []
    for i in range(len(resp)):
        result_list.append([resp[i]['elevation'], resp[i]['location']['lat'], resp[i]['location']['lng']])
    return result_list

def run_api_calls(name, lat_lon):
    print(f"getting data for {name}")
    calls = build_api_call_string(lat_lon)
    result_list = []
    for call in calls:
        result_list += get_elevations_lat_lon(call)
    return result_list

def get_distance_delta(point1: list, point2: list) -> float:
    return distance.distance(point1, point2).km * 1000 # return delta in meters 

def get_elevation_delta(point1_elev, point2_elev) -> float:
    return point2_elev - point1_elev

def get_rise_run_grade(seg_data: list[list]):
    seg_data_rrg = []
    for i in range(len(seg_data)-1):
        run = get_distance_delta(seg_data[i][1:3], seg_data[i+1][1:3])
        rise = get_elevation_delta(seg_data[i][0], seg_data[i+1][0])
        seg_data_rrg.append([run, rise, rise/run *100])
    return seg_data_rrg

In [2]:
# runner 
name, distance_seg, average_grade, maximum_grade, lat_lon  = retrieve_segment(id, access_token)
seg_data=run_api_calls(name, lat_lon)
get_rise_run_grade(seg_data)

getting data for Cumbre Del Sol - Full climb
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134


[[3.6516359538116387, 0.2578277587890625, 7.060609602113748],
 [11.360500806743698, 0.6875801086425781, 6.052374981870731],
 [20.30844732354519, 0.5357894897460938, 2.638259248529111],
 [20.990581313925667, 1.0349807739257812, 4.930691334589908],
 [15.6939349518112, 1.2223892211914062, 7.7889275375793074],
 [35.07031179438917, 3.6680831909179688, 10.459226061129055],
 [111.42235232426259, 14.180946350097656, 12.727200650752826],
 [12.175864064216437, 2.1521987915039062, 17.675942997992138],
 [39.28401398133909, 7.918876647949219, 20.15801300679428],
 [23.28361146307618, 6.0148162841796875, 25.832832220715225],
 [40.4981613621784, 10.389556884765625, 25.654391545954297],
 [25.499024567465046, 6.301361083984375, 24.712165233271183],
 [7.618287635716869, 1.7218017578125, 22.600902461862496],
 [18.571230812663792, 5.256278991699219, 28.30334211405602],
 [7.051101758923757, 1.5404129028320312, 21.846414298056477],
 [7.713651725403386, 2.6458816528320312, 34.30128487806033],
 [7.827407714267

In [11]:
distance.distance(seg_data[0][1:3], seg_data[1][1:3])

AttributeError: 'float' object has no attribute 'distance'

In [16]:
distance.distance((seg_data[1][1], seg_data[1][2]), (seg_data[0][1], seg_data[0][2]))

AttributeError: 'float' object has no attribute 'distance'

In [17]:
distance

3673.1